## 네이버 직방 크롤링 방식
- 세로 0.54
- 가로 0.145
- https://m.land.naver.com/map/37.6065609:126.9449408:14/VL/A1:B1:B2#mapFullList

1. 시작 위도 경도 기준점을 잡는다.
2. 한 화면에 해당하는 위도 경도 변화량이 얼마인지 측정한다.
3. 해당 변화량 만큼 이동한다.
4. 매물 목록을 누른다.
5. 첫번째 20개 매물의 주소를 가져온다.
6. 매물의 페이지 번호를 1씩 증가시킨다.
7. 마지막 페이지를 체크하는 불리언 값을 확인한다.
8. 해당 과정에서 모든 부동산 아이템 아이디를 획득한다.
9. 아이디를 바탕으로 크롤링을 수행한다.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, WebDriverException
)
import requests
import time
import json
import re
import math
import random
import os
import pandas as pd

In [ ]:
def crawl_item_ids(area_id, lat, lon, view, headless=False):
    """
    naver_crawling/
    │   # 해당 지역의 매물 리스트 페이지 번호와 수신 성공 여부 기록
    ├── page_list/
    │   ├── area1.csv
    │   ├── area2.csv
    │   └── area3.csv
    │   # 해당 지역의 매물 item_id와 수신 성공 여부 기록
    ├── id_list/
    │   ├── area1.csv
    │   ├── area2.csv
    │   └── area3.csv
    │   # 매물 상세정보 기록
    └── property_details/
        ├── 2500001.txt
        ├── 2500002.txt
        └── 2500003.txt
    """
    # 나중에 경로 외부에 배치할 것!!!
    # 경로 변수명으로 설정할 것!!
    # 파일 생성
    page_list_dir = "./page_list"
    page_list_csv_path = os.path.join(page_list_dir, f"{area_id}.csv")
    id_list_dir = "./id_list"
    id_list_csv_path = os.path.join(id_list_dir, f"{area_id}.csv")

    create_csv(
        dir=page_list_dir,
        csv_path=page_list_csv_path,
        columns=["page", "status"],
    )
    create_csv(
        dir=id_list_dir,
        csv_path=id_list_csv_path,
        columns=["item_id", "status"],
    )

    # 옵션 설정
    headers = set_headers()
    options = set_options(headless=headless)

    # 크롬 브라우저 자동 실행
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 10)

    # 접속 URL
    url = f"https://m.land.naver.com/map/{lat}:{lon}:{view}/VL/A1:B1:B2#mapFullList"
    driver.get(url)

    # 사이드바 열기
    open_sidebar(driver, wait)

    # 사이드바 스크롤 다운 n회 수행
    scroll_down_sidebar(driver, wait, num_scroll_down=3)
    
    # XHR 주소 수집
    item_ids_url = get_xhr_url(driver)

    # XHR 주소로 접근하면서 매물 아이디 수집
    num_pages = get_num_pages(item_ids_url)

    # 페이지 번호 기록
    page_list_df = pd.read_csv(page_list_csv_path)
    page_list_df["page"] = list(range(1, num_pages + 1))
    page_list_df.to_csv(page_list_csv_path, index=False)
    
    # item_id 크롤링
    # 나중에 페이지 번호를 순차적으로 진행하는 것이 아니라 랜덤 진행으로 변경할 수도 있음
    for i in range(1, num_pages + 1):
        # status가 success가 아닌 데이터에 대해 크롤링 진행
        if page_list_df.loc[i, "status"] == "success":
            continue

        # 랜덤 딜레이 2~10초
        delay_time = random.randint(2, 10)
        time.sleep(delay_time)

        try:
            # item_id 데이터 수신
            _url = item_ids_url.format(i)
            response = requests.get(_url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                articles = data.get("body", [])

                # item_ids 추출
                item_ids = []
                for article in articles:
                    if "atclNo" in article:
                        item_ids.append(article["atclNo"])

                # item_ids 기록 / 데이터가 많아지면 df 형식으로 저장 시 오래 걸림
                write_id_list(item_ids, id_list_csv_path)

                # 성공 status 기록
                update_status(
                    status="success",
                    column_name="page",
                    value=i, # 페이지 번호
                    csv_path=page_list_csv_path,
                )
                print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 item id 수신 완료 / url: {_url}")
            else:
                # 실패 status 기록
                update_status(
                    status="fail",
                    column_name="page",
                    value=i, # 페이지 번호
                    csv_path=page_list_csv_path,
                )
                print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 item id 수신 실패 / 상태 코드: {response.status_code}")
                
        except Exception as e:
            # 네트워크 중단 status 기록
            update_status(
                status="error",
                column_name="page",
                value=i, # 페이지 번호
                csv_path=page_list_csv_path,
            )
            print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 요청 중 에러 발생 → {e}")

        # 사이드바 스크롤 다운 n회 수행
        scroll_down_sidebar(driver, wait, num_scroll_down=5)

    # 브라우저 닫히지 않게 대기
    input("엔터를 누르면 브라우저를 종료합니다...")
    driver.quit()

def set_options(headless=False):
    # Chrome 옵션 설정
    options = Options()

    # 사용자 환경과 유사하게 브라우저 옵션 설정
    options.add_argument("--start-maximized")
    options.add_argument('--headless') if headless else None # 화면 비활성화
    options.add_argument('--disable-gpu')  # GPU 가속 비활성화 (Windows에서 필수일 수 있음)
    options.add_argument("--disable-infobars")  # 정보 표시창 제거 (자동화 감지 메시지 방지)
    options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 감지 기능 비활성화
    options.set_capability("goog:loggingPrefs", {"performance": "ALL"}) # DevTools 로그 수집을 위한 설정 / XHR 추적용도

    # 탐지 우회: Chrome 내부 플래그 설정 변경
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 플래그 제거
    options.add_experimental_option("useAutomationExtension", False)  # 셀레니움 확장 비활성화

    # User-Agent 변경
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )

    return options

def set_headers():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/120.0.0.0 Safari/537.36",

        "Referer": "https://m.land.naver.com/",  # 네이버 내부 요청처럼 위장

        "X-Requested-With": "XMLHttpRequest",    # AJAX 요청처럼 보이게

        "Accept": "application/json, text/javascript, */*; q=0.01",  # JSON 응답 허용

        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"     # 한국어 브라우저 설정 흉내
    }
    return headers

def open_sidebar(driver, wait):
    try:
        # 매물 목록 버튼 대기
        element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "btns_fixed_inner")))

        try:
            # 매물 목록 버튼 클릭
            article_button = element.find_element(By.CSS_SELECTOR, ".btn_option._article")

            # 버튼이 보이지 않을 경우 display 속성 강제로 block 설정
            if not article_button.is_displayed():
                element.execute_script("arguments[0].style.display = 'block';", article_button)
                WebDriverWait(driver, 5).until(EC.visibility_of(article_button)) # 렌더링 대기

            article_button.click()

        except NoSuchElementException:
            print("매물 목록 버튼을 찾을 수 없습니다.")
        
    except TimeoutException:
        print("매물 목록 버튼이 대기시간 안에 나타나지 않았습니다.")
    except WebDriverException as e:
        print(f"브라우저 오류 또는 네트워크 문제 발생: {e}")

def scroll_down_sidebar(driver, wait, num_scroll_down):
    # article_box 요소 찾기
    article_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".article_box.article_box--sale")))

    # n회 스크롤 내리기 수행
    for _ in range(num_scroll_down):
        driver.execute_script("arguments[0].scrollTop += 500;", article_box)
        time.sleep(0.5)

def get_xhr_url(driver):
    # XHR 로그 수집
    logs = driver.get_log("performance")

    # articleList 관련 요청 추출
    for entry in logs:
        log = json.loads(entry["message"])["message"]
        if (
            log.get("method") == "Network.requestWillBeSent"
            and "request" in log["params"]
            and "url" in log["params"]["request"]
        ):
            url = log["params"]["request"]["url"]
            if "articleList?itemId=" in url and "page=2" in url:
                print("[XHR URL]\n", url)
                formatted_url = url.replace("page=2", "page={}")
                return formatted_url
            
def get_num_pages(url):
    # totCnt 값 추출
    match = re.search(r"totCnt=(\d+)", url)
    tot_cnt = int(match.group(1))
    num_pages = math.ceil(tot_cnt / 20) # 한번에 20개의 매물을 보여줌

    return num_pages

def make_dir(folder_path):
    if not os.path.exists(folder_path):
        # 폴더가 없는 경우 생성
        os.makedirs(folder_path)
        print(f"폴더를 생성했습니다.: {folder_path}")
    else:
        # 폴더가 존재하는 경우 생성x
        print(f"폴더가 이미 존재합니다.: {folder_path}")

def create_csv(dir, csv_path, columns):
    # 폴더 생성
    make_dir(dir)

    # csv 파일 생성
    if not os.path.exists(csv_path):
        df = pd.DataFrame(columns=columns) # 빈 데이터프레임 생성
        df.to_csv(csv_path, index=False)
        print(f"csv 파일을 생성했습니다.: {csv_path}")
    else:
        print(f"csv 파일이 이미 존재합니다.: {csv_path}")
        
def update_status(status, column_name, value, csv_path):
    df = pd.read_csv(csv_path)
    df.loc[df[f"{column_name}"] == value, "status"] = status # 데이터 조회 후 저장
    df.to_csv(csv_path, index=False)

def write_id_list(item_ids, csv_path):
    # 속도 향상을 위해 pandas 대신 txt 기록 방식 선정
    with open(csv_path, "a", encoding="utf-8") as f: # a: append
        for id in item_ids:
            text = f"{id},\n" # status는 기록하지 않음
            f.write(text)

In [21]:
reference_lat = 37.6065609 # 기준 위도
reference_lon = 126.9449408 # 기준 경도
d_lat = 0.54
d_lon = 0.14

# 탐색 지역 좌표 생성
area_list = {}
cnt = 1
for lat_index in range(1):
    for lon_index in range(1):
        key = f"area{cnt}"
        area_list[key] = [
            lat_index * d_lat + reference_lat,
            lon_index * d_lon + reference_lon
        ]
        
print(100*"-")
print("탐색 지역 좌표 리스트")
print(area_list)
print(100*"-")

for area_id, (lat, lon) in area_list.items():
    print("탐색 지역 id:", area_id)
    print("탐색 지역 위도:", lat)
    print("탐색 지역 경도:", lon)
    
    crawl_item_ids(area_id, lat, lon)

----------------------------------------------------------------------------------------------------
탐색 지역 좌표 리스트
{'area1': [37.6065609, 126.9449408]}
----------------------------------------------------------------------------------------------------
탐색 지역 id: area1
탐색 지역 위도: 37.6065609
탐색 지역 경도: 126.9449408
폴더가 이미 존재합니다.: ./page_list
csv 파일을 생성했습니다.: ./page_list\area1.csv
폴더가 이미 존재합니다.: ./id_list
csv 파일을 생성했습니다.: ./id_list\area1.csv
[XHR URL]
 https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5798674&lft=126.8716843&top=37.6332448&rgt=127.0181973&totCnt=11696&cortarNo=&sort=rank&page=2


C:\Users\K\AppData\Local\Temp\ipykernel_21332\3109692087.py:256: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'success' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[f"{column_name}"] == value, "status"] = status # 데이터 조회 후 저장


[0001/0585] 0001 페이지 item id 수신 완료 / url: https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5798674&lft=126.8716843&top=37.6332448&rgt=127.0181973&totCnt=11696&cortarNo=&sort=rank&page=1
[0002/0585] 0002 페이지 item id 수신 완료 / url: https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5798674&lft=126.8716843&top=37.6332448&rgt=127.0181973&totCnt=11696&cortarNo=&sort=rank&page=2
[0003/0585] 0003 페이지 item id 수신 완료 / url: https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5798674&lft=126.8716843&top=37.6332448&rgt=127.0181973&totCnt=11696&cortarNo=&sort=rank&page=3
[0004/0585] 0004 페이지 item id 수신 완료 / url: https://m.land.naver.com/cluster/ajax/artic

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.178); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0032C7F3+24435]
	(No symbol) [0x002B2074]
	(No symbol) [0x001806E3]
	(No symbol) [0x00192321]
	(No symbol) [0x00191400]
	(No symbol) [0x00187B12]
	(No symbol) [0x00187BF2]
	(No symbol) [0x00185F1A]
	(No symbol) [0x001895C5]
	(No symbol) [0x00210225]
	(No symbol) [0x001ED7BC]
	(No symbol) [0x0020F20A]
	(No symbol) [0x001ED5B6]
	(No symbol) [0x001BC54F]
	(No symbol) [0x001BD894]
	GetHandleVerifier [0x006370A3+3213347]
	GetHandleVerifier [0x0064B0C9+3295305]
	GetHandleVerifier [0x0064558C+3271948]
	GetHandleVerifier [0x003C7360+658144]
	(No symbol) [0x002BB27D]
	(No symbol) [0x002B8208]
	(No symbol) [0x002B83A9]
	(No symbol) [0x002AAAC0]
	BaseThreadInitThunk [0x7684FCC9+25]
	R

In [ ]:


def crawl_property_datail(property_type, property_list_path, save_dir):
def crawl_property_detail(area_id, lat, lon, headless=False):
    """
    naver_crawling/
    │   # 해당 지역의 매물 리스트 페이지 번호와 수신 성공 여부 기록
    ├── page_list/
    │   ├── area1.csv
    │   ├── area2.csv
    │   └── area3.csv
    │   # 해당 지역의 매물 item_id와 수신 성공 여부 기록
    ├── id_list/
    │   ├── area1.csv
    │   ├── area2.csv
    │   └── area3.csv
    │   # 매물 상세정보 기록
    └── property_details/
        ├── 2500001.txt
        ├── 2500002.txt
        └── 2500003.txt
    """
    # 나중에 경로 외부에 배치할 것!!!
    # 경로 변수명으로 설정할 것!!
    # 파일 생성
    page_list_dir = "./page_list"
    page_list_csv_path = os.path.join(page_list_dir, f"{area_id}.csv")
    id_list_dir = "./id_list"
    id_list_csv_path = os.path.join(id_list_dir, f"{area_id}.csv")

    create_csv(
        dir=page_list_dir,
        csv_path=page_list_csv_path,
        columns=["page", "status"],
    )
    create_csv(
        dir=id_list_dir,
        csv_path=id_list_csv_path,
        columns=["item_id", "status"],
    )

    # 옵션 설정
    headers = set_headers()
    options = set_options(headless=headless)

    # 크롬 브라우저 자동 실행
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 10)

    # 접속 URL
    url = f"https://m.land.naver.com/map/{lat}:{lon}:14/VL/A1:B1:B2#mapFullList" # 나중에 view 레벨도 조정 가능한 버전으로 변경할 것
    driver.get(url)

    # 사이드바 열기
    open_sidebar(driver, wait)

    # 사이드바 스크롤 다운 n회 수행
    scroll_down_sidebar(driver, wait, num_scroll_down=3)
    
    # XHR 주소 수집
    item_ids_url = get_xhr_url(driver)

    # XHR 주소로 접근하면서 매물 아이디 수집
    num_pages = get_num_pages(item_ids_url)

    # 페이지 번호 기록
    page_list_df = pd.read_csv(page_list_csv_path)
    page_list_df["page"] = list(range(1, num_pages + 1))
    page_list_df.to_csv(page_list_csv_path, index=False)
    
    # item_id 크롤링
    # 나중에 페이지 번호를 순차적으로 진행하는 것이 아니라 랜덤 진행으로 변경할 수도 있음
    for i in range(1, num_pages + 1):
        # status가 success가 아닌 데이터에 대해 크롤링 진행
        if page_list_df.loc[i, "status"] == "success":
            continue

        # 랜덤 딜레이 2~10초
        delay_time = random.randint(2, 10)
        time.sleep(delay_time)

        try:
            # item_id 데이터 수신
            _url = item_ids_url.format(i)
            response = requests.get(_url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                articles = data.get("body", [])

                # item_ids 추출
                item_ids = []
                for article in articles:
                    if "atclNo" in article:
                        item_ids.append(article["atclNo"])

                # item_ids 기록 / 데이터가 많아지면 df 형식으로 저장 시 오래 걸림
                write_id_list(item_ids, id_list_csv_path)

                # 성공 status 기록
                update_status(
                    status="success",
                    column_name="page",
                    value=i, # 페이지 번호
                    csv_path=page_list_csv_path,
                )
                print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 item id 수신 완료 / url: {_url}")
            else:
                # 실패 status 기록
                update_status(
                    status="fail",
                    column_name="page",
                    value=i, # 페이지 번호
                    csv_path=page_list_csv_path,
                )
                print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 item id 수신 실패 / 상태 코드: {response.status_code}")
                
        except Exception as e:
            # 네트워크 중단 status 기록
            update_status(
                status="error",
                column_name="page",
                value=i, # 페이지 번호
                csv_path=page_list_csv_path,
            )
            print(f"[{i:04}/{num_pages:04}] {i:04} 페이지 요청 중 에러 발생 → {e}")

        # 사이드바 스크롤 다운 n회 수행
        scroll_down_sidebar(driver, wait, num_scroll_down=5)

    # 브라우저 닫히지 않게 대기
    input("엔터를 누르면 브라우저를 종료합니다...")
    driver.quit()

def set_options(headless=False):
    # Chrome 옵션 설정
    options = Options()

    # 사용자 환경과 유사하게 브라우저 옵션 설정
    options.add_argument("--start-maximized")
    options.add_argument('--headless') if headless else None # 화면 비활성화
    options.add_argument('--disable-gpu')  # GPU 가속 비활성화 (Windows에서 필수일 수 있음)
    options.add_argument("--disable-infobars")  # 정보 표시창 제거 (자동화 감지 메시지 방지)
    options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 감지 기능 비활성화
    options.set_capability("goog:loggingPrefs", {"performance": "ALL"}) # DevTools 로그 수집을 위한 설정 / XHR 추적용도

    # 탐지 우회: Chrome 내부 플래그 설정 변경
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 플래그 제거
    options.add_experimental_option("useAutomationExtension", False)  # 셀레니움 확장 비활성화

    # User-Agent 변경
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )

    return options

def set_headers():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/120.0.0.0 Safari/537.36",

        "Referer": "https://m.land.naver.com/",  # 네이버 내부 요청처럼 위장

        "X-Requested-With": "XMLHttpRequest",    # AJAX 요청처럼 보이게

        "Accept": "application/json, text/javascript, */*; q=0.01",  # JSON 응답 허용

        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"     # 한국어 브라우저 설정 흉내
    }
    return headers

def open_sidebar(driver, wait):
    try:
        # 매물 목록 버튼 대기
        element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "btns_fixed_inner")))

        try:
            # 매물 목록 버튼 클릭
            article_button = element.find_element(By.CSS_SELECTOR, ".btn_option._article")

            # 버튼이 보이지 않을 경우 display 속성 강제로 block 설정
            if not article_button.is_displayed():
                element.execute_script("arguments[0].style.display = 'block';", article_button)
                WebDriverWait(driver, 5).until(EC.visibility_of(article_button)) # 렌더링 대기

            article_button.click()

        except NoSuchElementException:
            print("매물 목록 버튼을 찾을 수 없습니다.")
        
    except TimeoutException:
        print("매물 목록 버튼이 대기시간 안에 나타나지 않았습니다.")
    except WebDriverException as e:
        print(f"브라우저 오류 또는 네트워크 문제 발생: {e}")

def scroll_down_sidebar(driver, wait, num_scroll_down):
    # article_box 요소 찾기
    article_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".article_box.article_box--sale")))

    # n회 스크롤 내리기 수행
    for _ in range(num_scroll_down):
        driver.execute_script("arguments[0].scrollTop += 500;", article_box)
        time.sleep(0.5)

def get_xhr_url(driver):
    # XHR 로그 수집
    logs = driver.get_log("performance")

    # articleList 관련 요청 추출
    for entry in logs:
        log = json.loads(entry["message"])["message"]
        if (
            log.get("method") == "Network.requestWillBeSent"
            and "request" in log["params"]
            and "url" in log["params"]["request"]
        ):
            url = log["params"]["request"]["url"]
            if "articleList?itemId=" in url and "page=2" in url:
                print("[XHR URL]\n", url)
                formatted_url = url.replace("page=2", "page={}")
                return formatted_url
            
def get_num_pages(url):
    # totCnt 값 추출
    match = re.search(r"totCnt=(\d+)", url)
    tot_cnt = int(match.group(1))
    num_pages = math.ceil(tot_cnt / 20) # 한번에 20개의 매물을 보여줌

    return num_pages

def make_dir(folder_path):
    if not os.path.exists(folder_path):
        # 폴더가 없는 경우 생성
        os.makedirs(folder_path)
        print(f"폴더를 생성했습니다.: {folder_path}")
    else:
        # 폴더가 존재하는 경우 생성x
        print(f"폴더가 이미 존재합니다.: {folder_path}")

def create_csv(dir, csv_path, columns):
    # 폴더 생성
    make_dir(dir)

    # csv 파일 생성
    if not os.path.exists(csv_path):
        df = pd.DataFrame(columns=columns) # 빈 데이터프레임 생성
        df.to_csv(csv_path, index=False)
        print(f"csv 파일을 생성했습니다.: {csv_path}")
    else:
        print(f"csv 파일이 이미 존재합니다.: {csv_path}")
        
def update_status(status, column_name, value, csv_path):
    df = pd.read_csv(csv_path)
    df.loc[df[f"{column_name}"] == value, "status"] = status # 데이터 조회 후 저장
    df.to_csv(csv_path, index=False)

def write_id_list(item_ids, csv_path):
    # 속도 향상을 위해 pandas 대신 txt 기록 방식 선정
    with open(csv_path, "a", encoding="utf-8") as f: # a: append
        for id in item_ids:
            text = f"{id},\n" # status는 기록하지 않음
            f.write(text)

- 성공 실패 기록을 해야함.
1. 찾는 좌표를 아이디화 한다.
2. 해당 아이디에 해당하는 매물 리스트(페이지)에 대한 성공 여부
    - 폴더명 : pagelist
    - 폴더 및 파일 구조
        - 좌표_id.csv
        - 내부 데이터 컬럼 id, success/fail
        - 페이지 번호 / 성공여부
    - 폴더명 : idlist
        - 좌표 id.csv
        - 컬럼명 id
        - 아이템 아이디
3. 아이템 아이디 리스트에 대한 성공여부

In [6]:

for lon_j in range(2):
    print(lon_j)

0
1


In [4]:
a = {
    'code': 'success',
    'hasPaidPreSale': False,
    'more': True,
    'TIME': False,
    'z': 14,
    'page': 1,
    'body': [{
        'atclNo': '2516285698',
        'cortarNo': '1130510300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '5/5',
        'prc': 38800,
        'rentPrc': 0,
        'hanPrc': '3억 8,800',
        'spc1': '59',
        'spc2': '51.18',
        'direction': '남향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_9/1743149041988fMity_JPEG/a89b03e95e87e2fa032e011f13523899.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.64286,
        'lng': 127.012457,
        'atclFetrDesc': '신축.대형3룸.남향.생애최초.디딤돌.보금자리.가오리역5분.3개동단지형',
        'tagList': ['4년이내', '화장실두개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516321088',
        'cortarNo': '1141011300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '5/6',
        'prc': 59000,
        'rentPrc': 0,
        'hanPrc': '5억 9,000',
        'spc1': '57',
        'spc2': '48.71',
        'direction': '남동향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_28/1743171841474J60ni_JPEG/465e4b50b816e91702a5613617a2361c.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.568707,
        'lng': 126.945304,
        'atclFetrDesc': '방3화2 베란다 주차.엘베.창고',
        'tagList': ['2년이내', '화장실두개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'rter',
        'cpNm': '알터',
        'cpCnt': 1,
        'rltrNm': '키부동산공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'rter',
            'mobileArticleLinkTypeCode': 'NONE',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': False
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516339428',
        'cortarNo': '1144012200',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'B1',
        'tradTpNm': '전세',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '5/5',
        'prc': 29000,
        'rentPrc': 0,
        'hanPrc': '2억 9,000',
        'spc1': '30',
        'spc2': '22.82',
        'direction': '남동향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_128/1743164281507N1nXu_JPEG/afd4bfb6c1d80701b5556bfa1b98ba35.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.549169,
        'lng': 126.905908,
        'atclFetrDesc': '완전신축 첫입주 한강뷰 2룸 희귀매물',
        'tagList': ['2년이내', '화장실한개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 6,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '21C02B1Na70d0ccf56d4bb0f237140f22705d9e6e358786299206f5e14231efc14c19883',
        'sameAddrMaxPrc': '2억 9,000',
        'sameAddrMinPrc': '2억 9,000',
        'cpid': 'woori',
        'cpNm': '우리집부동산',
        'cpCnt': 5,
        'rltrNm': '브이아이피부동산중개법인 주식회사',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'woori',
            'mobileArticleLinkTypeCode': 'NONE',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': False
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516221930',
        'cortarNo': '1144012200',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'B1',
        'tradTpNm': '전세',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '고/5',
        'prc': 29000,
        'rentPrc': 0,
        'hanPrc': '2억 9,000',
        'spc1': '31',
        'spc2': '23.39',
        'direction': '남향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_130/1743164521417jw7e7_JPEG/5df5b5543dfd472dead6478211676f6a.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.549169,
        'lng': 126.905908,
        'tagList': ['2년이내', '소형평수', '방한개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 9,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '21C02B1N86212cbb07fa01da1823c57cdc6708e9adb1a601c2b1be94b17ead84e2311156',
        'sameAddrMaxPrc': '2억 9,000',
        'sameAddrMinPrc': '2억 9,000',
        'cpid': 'woori',
        'cpNm': '우리집부동산',
        'cpCnt': 6,
        'rltrNm': '주식회사애플부동산중개법인',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'woori',
            'mobileArticleLinkTypeCode': 'NONE',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': False
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516284760',
        'cortarNo': '1130510300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 26900,
        'rentPrc': 0,
        'hanPrc': '2억 6,900',
        'spc1': '43',
        'spc2': '37.46',
        'direction': '서향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_126/17431490495732F3SH_JPEG/07daf8a1f166b1727871558d1828b898.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.64286,
        'lng': 127.012457,
        'atclFetrDesc': '신축.대형3룸.남향.생애최초.디딤돌.보금자리.가오리역5분.3개동단지형',
        'tagList': ['4년이내', '화장실한개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 2,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '25C02A1N5ca28253090ec42bca2bde2f878c59c74177c99768c2b479a5372b2b1be6c781',
        'sameAddrMaxPrc': '2억 6,900',
        'sameAddrMinPrc': '2억 6,900',
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516280878',
        'cortarNo': '1130510300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '4/5',
        'prc': 44000,
        'rentPrc': 0,
        'hanPrc': '4억 4,000',
        'spc1': '61',
        'spc2': '53.38',
        'direction': '남서향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_65/1743149049150he7rw_JPEG/820e98ea22af48a11e3509fffb73085d.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.646121,
        'lng': 127.011804,
        'atclFetrDesc': '신축.대형3룸.채광좋음.생애최초.디딤돌.보금자리.11자주식주차.',
        'tagList': ['4년이내', '화장실두개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516035831',
        'cortarNo': '1111018200',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '1/4',
        'prc': 110000,
        'rentPrc': 0,
        'hanPrc': '11억',
        'spc1': '155',
        'spc2': '84.84',
        'direction': '남향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_52/1743154630576lCHzU_JPEG/6d8759ee5892532168eba0028acb75e7.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.608085,
        'lng': 126.958319,
        'atclFetrDesc': '신축.대형3룸.단독테라스.주차100프로.남향.채광우수.고급자재마감.',
        'tagList': ['2년이내', '테라스', '1층'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 2,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '29C02A1N07cc827d619c57f1f29091aca10424e0874b7f2c9fb8cfa4966092aef265559f',
        'sameAddrMaxPrc': '11억',
        'sameAddrMinPrc': '9억',
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516035316',
        'cortarNo': '1111018200',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/4',
        'prc': 75000,
        'rentPrc': 0,
        'hanPrc': '7억 5,000',
        'spc1': '103',
        'spc2': '58.35',
        'direction': '서향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_78/1743154263362H4G9E_JPEG/8842dd1b0c09e5875d54289be2b17aa2.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.608085,
        'lng': 126.958319,
        'atclFetrDesc': '신축.복층2룸.주차100프로.채광우수.고급자재마감.',
        'tagList': ['2년이내', '테라스', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516291035',
        'cortarNo': '1138010700',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '저/9',
        'prc': 46500,
        'rentPrc': 0,
        'hanPrc': '4억 6,500',
        'spc1': '109',
        'spc2': '76.77',
        'direction': '남향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_215/1743154021619cAGHx_JPEG/212941e136ad04ea29e93525f6b0c84d.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.598989,
        'lng': 126.917814,
        'atclFetrDesc': '신축.첫입주3룸.넓은면적.생애최초가능.응암역3분.',
        'tagList': ['4년이내', '역세권', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516031432',
        'cortarNo': '1141011800',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 47000,
        'rentPrc': 0,
        'hanPrc': '4억 7,000',
        'spc1': '81',
        'spc2': '48.13',
        'direction': '동향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_225/17431541419282VqHc_JPEG/8a78f5a363e81f5d44cf08a3e69fc20e.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.586842,
        'lng': 126.934566,
        'atclFetrDesc': '신축.대형3룸.첫입주.시원한거실.트인전망.생애최초.디딤돌.특례보금자리.',
        'tagList': ['2년이내', '테라스', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516037896',
        'cortarNo': '1111018300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '3/5',
        'prc': 59000,
        'rentPrc': 0,
        'hanPrc': '5억 9,000',
        'spc1': '94',
        'spc2': '59.24',
        'direction': '남동향',
        'atclCfmYmd': '25.03.28.',
        'repImgUrl': '/20250328_179/1743164461536yrLXo_JPEG/c034eb47e95326dce4ac4f8958d33434.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.605288,
        'lng': 126.965192,
        'atclFetrDesc': '신축.넓은3룸.층수타입다양.생애최초.보금자리.디딤돌.',
        'tagList': ['2년이내', '화장실두개', '방세개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 2,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '21C02A1N2a981eb91882663b968da9f6ccaa2919c7081ea1123886b42e8df95be1bb67af',
        'sameAddrMaxPrc': '5억 9,500',
        'sameAddrMinPrc': '5억 9,000',
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516029182',
        'cortarNo': '1141011700',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '4/6',
        'prc': 65000,
        'rentPrc': 0,
        'hanPrc': '6억 5,000',
        'spc1': '101',
        'spc2': '59.94',
        'direction': '북향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_34/1743073862091Kr7DY_JPEG/9215aa12c3ecc1b12ea4a439198ea6c2.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.56669,
        'lng': 126.934621,
        'atclFetrDesc': '신축.대형3룸.단독대형테라스.연희IC.연희초인접.생애최초.보금자리.',
        'tagList': ['2년이내', '테라스', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516049136',
        'cortarNo': '1129013300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '6/9',
        'prc': 32000,
        'rentPrc': 0,
        'hanPrc': '3억 2,000',
        'spc1': '38',
        'spc2': '29.8',
        'direction': '남향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_151/1743068465235qAjNp_JPEG/5b9dc333f6dd32ccc3eb2b257f717281.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.604998,
        'lng': 127.011927,
        'atclFetrDesc': '신축.넓은2룸.안방붙박이장.생애최초.디딤돌.보금자리가능.정릉역3분.',
        'tagList': ['4년이내', '역세권', '화장실한개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516051822',
        'cortarNo': '1129013300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 41900,
        'rentPrc': 0,
        'hanPrc': '4억 1,900',
        'spc1': '66',
        'spc2': '57.07',
        'direction': '남동향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_50/17430682845127Rj2D_JPEG/d42dce465fe624b27075fb0c6f706a61.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.605125,
        'lng': 127.012696,
        'atclFetrDesc': '신축.넓은3룸.안방붙박이장.파우더룸.생애최초.디딤돌.보금자리.정릉역3분',
        'tagList': ['4년이내', '역세권', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516041718',
        'cortarNo': '1129013300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/4',
        'prc': 34000,
        'rentPrc': 0,
        'hanPrc': '3억 4,000',
        'spc1': '74',
        'spc2': '51.46',
        'direction': '남향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_121/1743077403404XXRDI_JPEG/ecf2938b1e70c1471c37ed13d85cc147.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.609595,
        'lng': 127.010922,
        'atclFetrDesc': '신축.가격좋은3룸.생애최초.특례보금자리.디딤돌가능.보국문역5분.',
        'tagList': ['4년이내', '역세권', '화장실두개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516070448',
        'cortarNo': '1129011500',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 51500,
        'rentPrc': 0,
        'hanPrc': '5억 1,500',
        'spc1': '55',
        'spc2': '47.29',
        'direction': '북서향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_119/1743061923417Cf0Bn_JPEG/6f3ac9d6d669942188fa8e10116b5edd.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.588833,
        'lng': 127.014758,
        'atclFetrDesc': '신축.넓은3룸.생애최초.특례보금자리.디딤돌가능.보문역.성신여대입구역.',
        'tagList': ['4년이내', '화장실두개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 2,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '21C02A1N0ef88ce4e5a5d6aa7fd2e5468df6b463feaf3ed80d3d87252eff5b910a691974',
        'sameAddrMaxPrc': '5억 1,500',
        'sameAddrMinPrc': '5억 1,500',
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516046608',
        'cortarNo': '1129013300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 25200,
        'rentPrc': 0,
        'hanPrc': '2억 5,200',
        'spc1': '31',
        'spc2': '23.85',
        'direction': '서향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_228/1743069842909aDABp_JPEG/da41a6dca7f6fcfc991dcf5ed894913d.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.607758,
        'lng': 127.007922,
        'atclFetrDesc': '신축.넓은2룸.디딤돌.보금자리.생애최초.넓은방크기.정릉역.정릉시장.',
        'tagList': ['4년이내', '화장실한개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516028856',
        'cortarNo': '1141011700',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '3/6',
        'prc': 56000,
        'rentPrc': 0,
        'hanPrc': '5억 6,000',
        'spc1': '85',
        'spc2': '50.61',
        'direction': '남향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_283/1743073865087ucalf_JPEG/75558d6f2fcb272a6b82c5fd7ef872eb.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.56669,
        'lng': 126.934621,
        'atclFetrDesc': '신축.넓은3룸.남향.채광좋음.연희IC.연희초인접.생애최초.보금자리.',
        'tagList': ['2년이내', '화장실두개', '방세개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516032866',
        'cortarNo': '1141011800',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/6',
        'prc': 37000,
        'rentPrc': 0,
        'hanPrc': '3억 7,000',
        'spc1': '61',
        'spc2': '40.89',
        'direction': '남서향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_248/1743073861763KeXyU_JPEG/7b25672a07177873ff2eedb8940afbf5.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.580553,
        'lng': 126.928721,
        'atclFetrDesc': '신축.가격좋은3룸.생애최초.디딤돌.보금자리.',
        'tagList': ['2년이내', '화장실한개', '소형평수'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 1,
        'sameAddrDirectCnt': 0,
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 1,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }, {
        'atclNo': '2516052218',
        'cortarNo': '1129013300',
        'atclNm': '빌라',
        'atclStatCd': 'R0',
        'rletTpCd': 'C02',
        'uprRletTpCd': 'C03',
        'rletTpNm': '빌라',
        'tradTpCd': 'A1',
        'tradTpNm': '매매',
        'vrfcTpCd': 'S_VR',
        'flrInfo': '2/5',
        'prc': 30000,
        'rentPrc': 0,
        'hanPrc': '3억',
        'spc1': '72',
        'spc2': '64.93',
        'direction': '서향',
        'atclCfmYmd': '25.03.27.',
        'repImgUrl': '/20250327_40/1743075121738WOWgh_JPEG/9e874b22b5d17d893fc1f03101719e60.JPG',
        'repImgTpCd': 'SITE',
        'repImgThumb': 'f130_98',
        'lat': 37.60812,
        'lng': 127.012643,
        'atclFetrDesc': '신축.넓은3룸.남향.정릉역7분.생애최초.특례보금자리.디딤돌가능.',
        'tagList': ['10년이내', '화장실한개', '방세개'],
        'bildNm': '',
        'minute': 0,
        'sameAddrCnt': 2,
        'sameAddrDirectCnt': 0,
        'sameAddrHash': '26C02A1Nd64f5974f56e0abeb47b6690c991043813d13ff2fd2e42cc7b4d302e105d5212',
        'sameAddrMaxPrc': '3억',
        'sameAddrMinPrc': '2억 9,900',
        'cpid': 'SERVE',
        'cpNm': '부동산써브',
        'cpCnt': 2,
        'rltrNm': '정도공인중개사사무소',
        'directTradYn': 'N',
        'minMviFee': 0,
        'maxMviFee': 0,
        'etRoomCnt': 0,
        'tradePriceHan': '',
        'tradeRentPrice': 0,
        'tradeCheckedByOwner': False,
        'cpLinkVO': {
            'cpId': 'SERVE',
            'mobileArticleUrl': 'https://www.serve.co.kr/redirect/nland?UID=',
            'mobileArticleLinkTypeCode': 'CPNAMEONLY',
            'mobileBmsInspectPassYn': 'Y',
            'pcArticleLinkUseAtArticleTitle': False,
            'pcArticleLinkUseAtCpName': False,
            'mobileArticleLinkUseAtArticleTitle': False,
            'mobileArticleLinkUseAtCpName': True
        },
        'dtlAddrYn': 'N',
        'dtlAddr': '',
        'isVrExposed': True,
        'vrUrl': 'https://fin.land.naver.comnull'
    }]
}

In [6]:
a.get("body")

[{'atclNo': '2516285698',
  'cortarNo': '1130510300',
  'atclNm': '빌라',
  'atclStatCd': 'R0',
  'rletTpCd': 'C02',
  'uprRletTpCd': 'C03',
  'rletTpNm': '빌라',
  'tradTpCd': 'A1',
  'tradTpNm': '매매',
  'vrfcTpCd': 'S_VR',
  'flrInfo': '5/5',
  'prc': 38800,
  'rentPrc': 0,
  'hanPrc': '3억 8,800',
  'spc1': '59',
  'spc2': '51.18',
  'direction': '남향',
  'atclCfmYmd': '25.03.28.',
  'repImgUrl': '/20250328_9/1743149041988fMity_JPEG/a89b03e95e87e2fa032e011f13523899.JPG',
  'repImgTpCd': 'SITE',
  'repImgThumb': 'f130_98',
  'lat': 37.64286,
  'lng': 127.012457,
  'atclFetrDesc': '신축.대형3룸.남향.생애최초.디딤돌.보금자리.가오리역5분.3개동단지형',
  'tagList': ['4년이내', '화장실두개', '소형평수'],
  'bildNm': '',
  'minute': 0,
  'sameAddrCnt': 1,
  'sameAddrDirectCnt': 0,
  'cpid': 'SERVE',
  'cpNm': '부동산써브',
  'cpCnt': 1,
  'rltrNm': '정도공인중개사사무소',
  'directTradYn': 'N',
  'minMviFee': 0,
  'maxMviFee': 0,
  'etRoomCnt': 0,
  'tradePriceHan': '',
  'tradeRentPrice': 0,
  'tradeCheckedByOwner': False,
  'cpLinkVO': {'cpId': 'S

In [8]:
xhr_url = "https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5504084&lft=126.8736155&top=37.662671&rgt=127.0162661&totCnt=19626&cortarNo=&sort=rank&page=2"
formatted_url = xhr_url.replace("page=2", "page={page}")

In [10]:
formatted_url.format(page=1)

'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL%3AYR%3ADSD&tradTpCd=A1%3AB1%3AB2&z=14&lat=37.6065609&lon=126.9449408&btm=37.5504084&lft=126.8736155&top=37.662671&rgt=127.0162661&totCnt=19626&cortarNo=&sort=rank&page=1'

In [16]:
13206 / 20

660.3

In [12]:
text = "https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL:YR:DSD&tradTpCd=A1:B1:B2&z=14&lat=37.5882&lon=127.0464&btm=37.5623844&lft=126.9731435&top=37.6140067&rgt=127.1196565&totCnt=13236&cortarNo=&sort=rank&page=2"

a = text.split("&")
print(a)

['https://m.land.naver.com/cluster/ajax/articleList?itemId=', 'mapKey=', 'lgeo=', 'showR0=', 'rletTpCd=VL:YR:DSD', 'tradTpCd=A1:B1:B2', 'z=14', 'lat=37.5882', 'lon=127.0464', 'btm=37.5623844', 'lft=126.9731435', 'top=37.6140067', 'rgt=127.1196565', 'totCnt=13236', 'cortarNo=', 'sort=rank', 'page=2']


In [ ]:
https://new.land.naver.com/api/articles/2515589012?complexNo=

In [12]:
import requests

url = "https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL:YR:DSD&tradTpCd=A1:B1:B2&z=14&lat=37.5882&lon=127.0464&btm=37.5623844&lft=126.9731435&top=37.6140067&rgt=127.1196565&totCnt=13236&cortarNo=&sort=rank&page=2"
# url = "https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=VL:YR:DSD&tradTpCd=A1:B1:B2&z=14&lat=37.5882&lon=127.0464&cortarNo=&sort=rank&page=2"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://m.land.naver.com/",
    "X-Requested-With": "XMLHttpRequest",  # ✅ AJAX 요청처럼 보이게
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive"
}

# 세션을 사용하면 쿠키 자동 관리됨 (크롤링 시 더 안정적)
session = requests.Session()

# 실제 요청
response = session.get(url, headers=headers)

# 응답 확인
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"❌ 요청 실패: {response.status_code}")


{'code': 'success', 'paidPreSale': {'preSaleComplexNumber': 6027884, 'preSaleComplexName': '이문아이파크자이 3-2BL', 'preSaleAddress': '서울시 동대문구 이문동', 'preSaleAddressSector': '이문동', 'preSaleDetailAddress': '412-1번지 일대', 'preSaleStageCode': 'C13', 'preSaleStageDetails': '2023.10.20', 'preSaleTypeCode': 'IA01', 'preSaleFormCode': '11', 'occupancyYearMonth': '2026.05', 'thumbnail': 'https://landthumb-phinf.pstatic.net/20250107_61/1736234403024OsQNP_JPEG/uploadfile_202501075270603.jpg', 'featureMarkTypeCode': '2', 'minPreSalePrice': 974150000, 'maxPreSalePrice': 1430770000, 'minPreSaleArea': 78.5, 'maxPreSaleArea': 128.55, 'totalHouseholdsNumber': 152, 'preSaleHouseholdsNumber': 0, 'xcoordinate': 127.056163, 'ycoordinate': 37.600839, 'preSaleDetailsPageURL': 'https://isale.land.naver.com/NewiSaleMobile/Home/#SYDetail?build_dtl_cd=6027884&supp_cd=9043497'}, 'hasPaidPreSale': True, 'more': True, 'TIME': False, 'z': 14, 'page': 2, 'body': [{'atclNo': '2515775152', 'cortarNo': '1129011400', 'atclNm': 

In [14]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# ✅ Chrome 옵션 + 봇 방지 코드 설정
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)

# ✅ DevTools 로그 캡처 설정 (이제는 set_capability로!)
options.set_capability("goog:loggingPrefs", {"performance": "ALL"})

# ✅ 드라이버 실행
driver = webdriver.Chrome(options=options)

# # ✅ WebDriver 탐지 우회용 JS 주입
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
#     "source": """
#         Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
#         window.navigator.chrome = { runtime: {} };
#         Object.defineProperty(navigator, 'languages', { get: () => ['en-US', 'en'] });
#         Object.defineProperty(navigator, 'plugins', { get: () => [1, 2, 3] });
#     """
# })

# ✅ 페이지 접속
driver.get("https://m.land.naver.com")

# ✅ XHR 로그 수집
logs = driver.get_log("performance")

# ✅ XHR 요청 중 articleList URL 출력
for entry in logs:
    log = json.loads(entry["message"])["message"]
    if (
        log.get("method") == "Network.requestWillBeSent"
        and "request" in log["params"]
        and "url" in log["params"]["request"]
    ):
        url = log["params"]["request"]["url"]
        if "articleList" in url:
            print("[📦 XHR URL]", url)

driver.quit()


## 봇 감지 테스트 코드

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Chrome 옵션 설정
options = Options()

# 사용자 환경과 유사하게 브라우저 옵션 설정
options.add_argument("--start-maximized")
options.add_argument("--disable-infobars")  # 정보 표시창 제거 (자동화 감지 메시지 방지)
options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 감지 기능 비활성화

# 탐지 우회: Chrome 내부 플래그 설정 변경
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 플래그 제거
options.add_experimental_option("useAutomationExtension", False)  # 셀레니움 확장 비활성화

# User-Agent 변경 (기본 user-agent는 봇으로 감지될 수 있음)
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

# 드라이버 실행
driver = webdriver.Chrome(options=options)

# # 자바스크립트 실행: `navigator.webdriver` 제거
# # 많은 봇 탐지 시스템은 이 속성이 `true`인 걸로 셀레니움을 감지함
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
#     "source": """
#         Object.defineProperty(navigator, 'webdriver', {
#             get: () => undefined
#         });
#         window.navigator.chrome = {
#             runtime: {}
#         };
#         Object.defineProperty(navigator, 'languages', {
#             get: () => ['en-US', 'en']
#         });
#         Object.defineProperty(navigator, 'plugins', {
#             get: () => [1, 2, 3, 4, 5]
#         });
#     """
# })

# 페이지 접속 (예: 봇 탐지 여부 확인용)
driver.get("https://bot.sannysoft.com/")  # 봇 탐지 여부 보여주는 사이트

# 잠시 대기 후 종료
time.sleep(10)
driver.quit()

In [ ]:
import requests
import time
import random
import pandas as pd
from tqdm import tqdm

def crawl_item_ids(property_type, geohashs):

    if property_type == "villa":
        url_format = "https://apis.zigbang.com/v2/items/villa?geohash={}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&salesTypes[2]=매매&salesPriceMin=0&domain=zigbang&checkAnyItemWithoutFilter=true"
    elif property_type == "apt":
        pass

    property_list = []
    num_geohashs = len(geohashs)
    for i, geohash in enumerate(geohashs):
        api_url = url_format.format(geohash)

        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            property_list.append(data)
            print(f"[{i+1:04}/{num_geohashs:04}] {geohash} 구역 데이터 수신 완료")
        else:
            # 성공 혹은 실패 시 재수행을 위해 성공 실패 지역 이름을 기록하는 csv와 코드 필요
            # 여기에도 재실행 및 예외처리 적용
            print(f"[{i+1:04}/{num_geohashs:04}] {geohash} 구역 데이터 수신 실패 / 상태 코드: {response.status_code}")

        delay_time = random.randint(2, 10)
        time.sleep(delay_time)

    return property_list

In [ ]:
# https://new.land.naver.com/api/developmentplan/rail/list?zoom=15&leftLon=126.9290436&rightLon=127.0016564&topLat=37.5499007&bottomLat=37.5277817